In [ ]:
AICORE_SERVICE_KEY = ""  # @param {type:"string"}

groupID = ""
scenario = "AI-161-"+groupID
targetModel = "gpt-4o-mini:latest"

# Dependencies and Helper Functions

## AI Core Setup

In [22]:
import json

def fetch_credentials(resource_group: str = "default"):
    ai_core_key = json.loads(AICORE_SERVICE_KEY)
    return {
        "base_url": ai_core_key['serviceurls']['AI_API_URL'],
        "auth_url": ai_core_key["url"],
        "client_id": ai_core_key["clientid"],
        "client_secret": ai_core_key["clientsecret"],
        "resource_group": resource_group

    }



In [23]:
%pip install rich PyYAML sap-ai-sdk-gen

Looking in indexes: https://pypi.org/simple, https://int.repositories.cloud.sap/artifactory/api/pypi/proxy-deploy-releases-hyperspace-pypi/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
from gen_ai_hub.proxy import get_proxy_client, GenAIHubProxyClient
import pathlib
import yaml

from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.label import Label
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from gen_ai_hub.orchestration.service import OrchestrationService

client = GenAIHubProxyClient(**fetch_credentials())
deployments = client.ai_core_client.deployment.query()
orchestration_deployment = None
for deployment in deployments.resources:
    if deployment.scenario_id == "orchestration":
        orchestration_deployment = deployment
    break

In [25]:
import requests

def check_model_availability(latest_only: bool = True, filter_list = None):

    url = client.ai_core_client.base_url + "/lm/scenarios/foundation-models/models"
    response = requests.get(url=url, headers=client.request_header)
    models = {}
    for model in response.json()['resources']:
        if  not any([scenario["executableId"] == "orchestration" for scenario in model["allowedScenarios"]]):
            continue
        if filter_list is not None and model["model"] not in filter_list:
            continue
        models[model["model"]] = []
        for version in model["versions"]:
            if not any([v == "text-generation" for v in version.get("capabilities", [])]):
                continue
            if not latest_only or version["isLatest"]:
                models[model["model"]].append(version["name"])
            if version["isLatest"]:
                models[model["model"]].append("latest")

    return [f'{k}:{v}' for k, vv in models.items() for v in vv if len(vv) > 0 ]

filter_list=[
    'gpt-5',
    'gpt-5-mini',
    'gpt-5-nano',
    'gpt-4.1',
    'gpt-4.1-mini',
    'gpt-4.1-nano',
    'gpt-4o',
    'gpt-4o-mini',
    'gemini-2.5-pro',
    'gemini-2.5-flash',
    'anthropic--claude-4-sonnet',
    'anthropic--claude-4-opus'
]

SUPPORTED_MODELS = check_model_availability(filter_list=filter_list)


## Helper Functions Prompt Management

In [26]:
from logging import PlaceHolder
from pydantic import BaseModel
from typing import List
import re
import requests
import json


from gen_ai_hub.prompt_registry.models.prompt_template import PromptTemplateSpec as PromptTemplateSpec_
from gen_ai_hub.prompt_registry.models.prompt_template import PromptTemplate


from rich.console import Console
from rich.highlighter import RegexHighlighter
from rich.theme import Theme
from rich.panel import Panel
from rich import print

class PromptTemplateSpec(BaseModel):
    template: List[PromptTemplate]


    @property
    def placeholders(self):
        placeholders = set()
        pattern = re.compile(r'\{\{\s*\?\s*(\w+)\s*\}\}')
        for message in self.template:
            placeholders.update(pattern.findall(message.content))
        return placeholders

    @classmethod
    def from_optimizer_result(cls, input_):
        placeholders = input_.get("user_message_template_fields", [])
        def replace(msg):
            for key in placeholders:
                msg = msg.replace("{"+key+"}", "{{?"+ key + "}}")
            return msg

        return cls(
            template=[
                {
                    "role": "system",
                    "content": replace(input_["system_prompt"]),
                },{
                    "role": "user",
                    "content": replace(input_["user_message_template"]),
                }
            ]
        )

    def escape_curly_brackets(self) -> "PromptTemplateSpec":
        # 1. Hide each {{?key}} placeholder with a unique token
        placeholder_pattern = re.compile(r'\{\{\s*\?\s*(\w+)\s*\}\}')
        mapping = {}
        counter = 1

        def _hide(match):
            nonlocal counter
            token = f"__PLACEHOLDER_{counter}__"
            mapping[token] = match.group(0)
            counter += 1
            return token

        new_templates = []
        for msg in self.template:
            # a) hide custom placeholders
            hidden = placeholder_pattern.sub(_hide, msg.content)
            # b) escape all remaining braces
            escaped = hidden.replace('{', '{{').replace('}', '}}')
            # c) restore the original placeholders
            print(mapping)
            for token, original in mapping.items():
                escaped = escaped.replace(token, original)

            new_templates.append(PromptTemplate(role=msg.role, content=escaped))

        # return a fresh copy
        return PromptTemplateSpec(template=new_templates)



def fetch_prompt_template(prompt_template: str) -> PromptTemplateSpec:
    headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    url = f"{client.ai_core_client.base_url}/lm/promptTemplates"
    scenario, sep, name = prompt_template.partition("/")
    if sep:
        name, sep, version = name.partition(":")
    if sep:
        body = {"name": name,
                "version": version,
                "scenario": scenario,
                "includeSpec": True
            }
        response =  requests.get(url, headers=headers, params=body)
        response.raise_for_status()
        response = response.json()
        if response["count"] > 0:
            response = response["resources"][0]
        else:
            raise ValueError(f"Prompt template {name} not found.")
    else:
        url += f"/{prompt_template}"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        response = response.json()
    return PromptTemplateSpec.model_validate(response["spec"])

def load_prompt_template(prompt: str | pathlib.Path | list | dict | PromptTemplateSpec) -> PromptTemplateSpec:
    if isinstance(prompt, PromptTemplateSpec):
        return prompt
    if isinstance(prompt, PromptTemplateSpec_):
        return prompt
    if isinstance(prompt, (str, pathlib.Path)) and pathlib.Path(prompt).exists():
        with open(prompt, "r") as f:
            prompt = yaml.safe_load(f)
    elif isinstance(prompt, str):
        return fetch_prompt_template(prompt)
    if isinstance(prompt, dict):
        # expect dict with keys "system" [optional] and "user"
        messages = []
        if "system" in prompt:
            messages.append({"role": "system", "content": prompt["system"]})
        messages.append({"role": "user", "content": prompt["user"]})
        return PromptTemplateSpec(template=messages)
    elif isinstance(prompt, list):
        # expect list of dicts with keys "role" and "content"
        return PromptTemplateSpec(template=messages)
    else:
        raise ValueError("Prompt must be a string, Path, list or dict")


def push_prompt_template(prompt_template: PromptTemplateSpec,
                         prompt_template_name_registry: str,
                         prompt_template_version: str,
                         scenario: str,
                         update=False):
    headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    url = f"{client.ai_core_client.base_url}/lm/promptTemplates"
    body = {"name": prompt_template_name_registry,
            "version": prompt_template_version,
            "scenario": scenario}
    res = requests.get(url, headers=headers, params=body)
    res.raise_for_status()
    res = res.json()
    if res["count"] > 0 and not update:
        print(f"Prompt template {prompt_template_name_registry} already exists. Use update=True to update.")
        return res["resources"][0]
    # Prepare body

    body["spec"] = prompt_template.model_dump()
    # Prepare headers
    response = requests.post(url, headers=headers, json=body)
    # Handle response
    if response.status_code == 201:
        response = response.json()
    elif response.status_code in (400, 409, 413):
        # Return error details
        raise requests.HTTPError(f"Upload failed ({response.status_code}): {response.text}")
    else:
        response.raise_for_status()
    return response.json()



import re

def convert_py_notation(template):
    pattern = re.compile(r'\{\{\s*\?\s*(\w+)\s*\}\}')
    return pattern.sub(lambda match: "{" + match.group(1) + "}", template)


def validate_prompt(prompt: PromptTemplateSpec):
    values = {k: "???" for k in prompt.placeholders}

    for message in prompt.template:
        if message.role == "user":
            try:
                convert_py_notation(message.content).format(**values)
            except KeyError as err:
                msg = ["Unexpected key error when running test formatting."]
                msg += ["This is most likeyly due to unescaped curly brackets."]
                msg += ["You can try fixing this by running `prompt = prompt.escape_curly_brackets()` and use the new prompt template."]
                raise ValueError("\n".join(msg)) from err
    return True




class TemplateHighlighter(RegexHighlighter):
    """Apply style to anything that looks like an email."""

    base_style = "template."
    highlights = [r"(?P<placeholder>\{\{\s*\?[^\{\}\s]+\s*\}\})"]

highlighter = TemplateHighlighter()
theme = Theme({"template.placeholder": "bold magenta", "example.email": "bold magenta"})
console = Console(highlighter=highlighter, theme=theme)


def print_prompt_template(prompt_template: PromptTemplateSpec | str | pathlib.Path, addition: str | None = None):

    prompt_template = load_prompt_template(prompt_template)
    addition = f' - {addition}' if addition else ''

    for message in prompt_template.template:
        if message.role == "system":
            console.print(Panel(highlighter(message.content), title="System Message" + addition, border_style="red"))
        elif message.role == "user":
            console.print(Panel(highlighter(message.content), title="User Message" + addition, border_style="green"))
        else:
            console.print(Panel(highlighter(message.content), title="Assistant Message" + addition))



## Helper Function Dataset and Artifact Handling

In [27]:
from typing import List
import requests
import mimetypes
from urllib.parse import quote
import pathlib
import json


def validate_dataset(dataset: str | pathlib.Path | list, expected_keys: None | List[str] = None) -> bool:
    if isinstance(dataset, (str, pathlib.Path)):
        with open(dataset, "r") as f:
            try:
                dataset = json.load(f)
            except json.JSONDecodeError as e:
                raise ValueError(f"Invalid JSON in file: {e}")
    if not isinstance(dataset, list):
        raise ValueError("Dataset must be a list of dictionaries.")

    def validate_item(item: dict, excepted_keys: None | List[str]) -> bool:
        excepted_keys = set(excepted_keys) if excepted_keys else None
        if set(item.keys()) != {"fields", "answer"}:
            raise ValueError("Each item must contain 'fields' and 'answer' keys.")
        if not isinstance(item["fields"], dict):
            raise ValueError("'fields' must be a dictionary.")
        fields = set(item["fields"].keys())
        if excepted_keys is not None:
            if fields != excepted_keys:
                if fields.difference(excepted_keys):
                    raise ValueError(f"Unexpected keys in 'fields'. Expected: {excepted_keys}, Found: {fields}")
                if excepted_keys.difference(fields):
                    raise ValueError(f"Missing keys in 'fields'. Expected: {excepted_keys}, Found: {fields}")
        if not all([isinstance(k, str) for k in item["fields"].values()]):
            raise ValueError("All values in 'fields' must be strings.")
        return fields

    excepted_keys = expected_keys
    for i, item in enumerate(dataset):
        if not isinstance(item, dict):
            raise ValueError("Each item in the dataset must be a dictionary.")
        try:
            excepted_keys = validate_item(item, excepted_keys)
        except ValueError as e:
            raise ValueError(f"Error in entry {i}") from e
    return True


def upload_file(secret: str,
                local_path: str | pathlib.Path,
                remote_path: str,
                overwrite: bool = False,) -> str:
    # check if secret exists
    secrets = [r.name for r in client.ai_core_client.object_store_secrets.query().resources]
    if secret not in secrets:
        raise ValueError(f"Secret '{secret}' not found in object store secrets. Known secrets: {secrets}")

    # Check if local path exists
    remote_path = remote_path.lstrip("/")
    if "/" not in remote_path and not allow_bucket_root:
        raise ValueError(
            "Remote path must use subdirectories. Otherwise the whole bucket will be used as an input artifact. Set allow_bucket_root=True to allow this."
        )

    # URL-encode the path parameter
    path = f"{secret}/" + remote_path.lstrip("/")
    encoded_path = quote(path, safe="")
    url = f"{client.ai_core_client.base_url}/lm/dataset/files/{encoded_path}"
    params = {"overwrite": str(overwrite).lower()}

    # Prepare headers
    headers = {
        **client.request_header,
        "Content-Type": "application/octet-stream",
    }
    # Guess MIME type
    guessed_type, _ = mimetypes.guess_type(local_path)
    if guessed_type:
        headers["Content-Type"] = guessed_type

    with open(local_path, "rb") as f:
        response = requests.put(url, params=params, headers=headers, data=f)

    # Handle response
    if response.status_code == 201:
        response = response.json()
    elif response.status_code in (400, 409, 413):
        # Return error details
        raise requests.HTTPError(f"Upload failed ({response.status_code}): {response.text}")
    else:
        response.raise_for_status()
    return "/".join(response["url"].split("/")[:-1])

def register_artifact(artifact_name:str,
                      artifact_root_path,
                      scenario: str,
                      artifact_kind: Artifact.Kind,
                      description: str):
    headers = {
        **client.request_header,
        "Content-Type": "application/octet-stream",
    }

    for artifact in client.ai_core_client.artifact.query().resources:
        if artifact_root_path==(artifact.url + "/"):
            return artifact 

    # Create new artifact
    new_artifact = client.ai_core_client.artifact.create(
        name=artifact_name,
        kind=artifact_kind,
        url=artifact_root_path,
        scenario_id=scenario,
        description=description,
        resource_group=headers[client.ai_core_client.rest_client.resource_group_header]
    )
    return new_artifact

## Helper Function Execution Result Handling

In [28]:
from rich.console import Console
from rich.table import Table

def fetch_results(execution_id, file_name: str='result-data/results.json'):
    response = client.ai_core_client.execution.get(execution_id = execution_id)
    if response.status.name not in {'DEAD', 'COMPLETED'}:
        raise RuntimeError('Execution not finished!')
    path = f"default/{execution_id}/{file_name}"
    encoded_path = quote(path, safe="")
    url = f"{client.ai_core_client.base_url}/lm/dataset/files/{encoded_path}"
    headers = {
        **client.request_header,
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()# results = response.json()


def print_result(result):
    origin_model = result["origin_model"]
    table = Table(title="Performance")
    table.add_column("Model", justify="right", style="cyan", no_wrap=True)
    table.add_column("Pre Optimization", style="magenta")
    table.add_column("Post Optimization", justify="right", style="green")
    table.add_row(origin_model["model_name"], f'{origin_model["score"]:.3f}', "n/a - reference run")
    for m in result["target_models"]:
        table.add_row(m["model_name"], f'{m["pre_optimization_score"]:.3f}', f'{m["post_optimization_score"]:.3f}')
    console.print(table)
    for m in result["target_models"]:
        print_prompt_template(PromptTemplateSpec.from_optimizer_result(m), addition=m['model_name'])



In [29]:
import time
from datetime import datetime, timedelta, timezone
from typing import Callable, Optional

from rich.console import Console
from rich.live import Live
from rich.panel import Panel
from rich.spinner import Spinner
from rich.table import Table
from rich.text import Text
from requests import HTTPError

console = Console()

# Add ERROR, use lowercase color names (Rich style)
_STATUS_COLORS = {
    "UNKNOWN": "bright_black",
    "PENDING": "yellow",
    "RUNNING": "cyan",
    "COMPLETED": "green",
    "ERROR": "red",
    "DEAD": "red",
    "STOPPED": "red",
}

# Define terminal statuses (polling stops on any of these)
_TERMINAL_STATUSES = {"COMPLETED", "ERROR", "STOPPED", "DEAD"}


def _render_status_panel(
    execution_id: str,
    status: str,
    created_at: datetime,
    note: Optional[str] = None,
) -> Panel:
    # Ensure created_at is timezone-aware in UTC
    if created_at.tzinfo is None:
        created_at = created_at.replace(tzinfo=timezone.utc)

    now = datetime.now(timezone.utc)
    elapsed = now - created_at

    table = Table.grid(padding=(0, 1))
    table.add_column(justify="right", style="bold")
    table.add_column()

    color = _STATUS_COLORS.get(status, "white")
    table.add_row("Execution ID:", Text(execution_id, style="bold"))
    table.add_row("Status:", Text(status, style=f"bold {color}"))
    table.add_row("Elapsed:", Text(str(elapsed).split(".")[0]))  # drop microseconds

    if note:
        table.add_row("Note:", Text(note, style="italic"))

    # Show spinner only for non-terminal states
    if status not in _TERMINAL_STATUSES:
        spinner = Spinner("dots", text="Polling… (Interrupt cell to stop)")
        content = Table.grid()
        content.add_row(spinner)
        content.add_row(table)
        return Panel(content, title="Run Status", border_style=color)
    else:
        return Panel(table, title="Run Status", border_style=color)


def poll_status(
    execution_id: str,
    client,
    *,
    interval: float = 30.0,
    timeout: Optional[float] = None,
    cancel_callback: Optional[Callable[[str], None]] = None,
) -> str:
    """
    Block until the execution reaches a terminal state (COMPLETED, ERROR, STOPPED, DEAD),
    rendering a live status panel. Interrupt the cell to stop polling; cancel the remote
    run via your client if you also want to unblock the run.
    """
    # Get initial execution info (for created_at)
    first_resp = client.ai_core_client.execution.get(execution_id=execution_id)
    created_at_raw = first_resp.created_at
    if not created_at_raw:
        raise ValueError("Execution response did not include 'created_at'.")
    created_at = (
        created_at_raw if isinstance(created_at_raw, datetime)
        else datetime.fromisoformat(created_at_raw.replace("Z", "+00:00"))
    )

    note = (
        "This cell is blocking until the run completes or stops. "
        "To stop polling, interrupt the cell. "
        "If you also want to stop the remote run, cancel it via your client."
    )

    start_poll = time.monotonic()
    with Live(console=console, transient=False, refresh_per_second=8) as live:
        try:
            while True:
                resp = client.ai_core_client.execution.get(execution_id=execution_id)
                status = resp.status.name  # expecting an Enum-like object with .name

                live.update(_render_status_panel(execution_id, status, created_at, note=note))

                # Terminal states
                if status in _TERMINAL_STATUSES:
                    if status == "COMPLETED":
                        console.print(Panel(Text("Run completed successfully ✅", style="bold green"), border_style="green"))
                    elif status == "ERROR":
                        console.print(Panel(Text("Run ended in ERROR ❌", style="bold red"), border_style="red"))
                    elif status == "STOPPED":
                        console.print(Panel(Text("Run was STOPPED ⏹️", style="bold red"), border_style="red"))
                    elif status == "DEAD":
                        console.print(Panel(Text("Run is DEAD 💀", style="bold red"), border_style="red"))
                    return status

                # Timeout?
                if timeout is not None and (time.monotonic() - start_poll) > timeout:
                    raise TimeoutError(
                        f"Polling timed out after {timeout} seconds (last status: {status}). "
                        "You can interrupt the cell to stop polling, and cancel the remote run if needed."
                    )

                time.sleep(max(0.05, float(interval)))

        except KeyboardInterrupt:
            console.print()  # newline
            console.rule("[bold]Polling interrupted by user[/bold]")
            console.print(
                "- Polling has been stopped.\n",
                style="italic",
            )
            if cancel_callback:
                try:
                    cancel_callback(execution_id)
                    console.print("Attempted to cancel the remote run via provided cancel_callback.", style="bold yellow")
                except Exception as e:
                    console.print(f"Cancel callback raised an error: {e}", style="red")
            return "INTERRUPTED"


## Helper Function Prompt Optimizer

In [71]:
import requests

def check_model_availability(latest_only: bool = True, filter_list = None):

    url = client.ai_core_client.base_url + "/lm/scenarios/foundation-models/models"
    response = requests.get(url=url, headers=client.request_header)
    models = {}
    for model in response.json()['resources']:
        if  not any([scenario["executableId"] == "orchestration" for scenario in model["allowedScenarios"]]):
            continue
        if filter_list is not None and model["model"] not in filter_list:
            continue
        models[model["model"]] = []
        for version in model["versions"]:
            if not any([v == "text-generation" for v in version.get("capabilities", [])]):
                continue
            if not latest_only or version["isLatest"]:
                models[model["model"]].append(version["name"])
            if version["isLatest"]:
                models[model["model"]].append("latest")

    return [f'{k}:{v}' for k, vv in models.items() for v in vv if len(vv) > 0 ]

filter_list=[
    'gpt-5',
    'gpt-5-mini',
    'gpt-5-nano',
    'gpt-4.1',
    'gpt-4.1-mini',
    'gpt-4.1-nano',
    'gpt-4o',
    'gpt-4o-mini',
    'gemini-2.5-pro',
    'gemini-2.5-flash',
    'anthropic--claude-4-sonnet',
    'anthropic--claude-4-opus'
]

SUPPORTED_MODELS = check_model_availability(filter_list=filter_list)

In [77]:
import re

SEMVER_PATTERN = re.compile(r'^[a-zA-Z0-9_-]+:[0-9]+\.[0-9]+\.[0-9]+$')
SUPPORTED_METRICS = ["LLMaaJ:Sem_Sim_1", "JSON_Match"]

def create_optim_config(metric: str,
                  reference_model: str,
                  targets: dict,
                  dataset_path: str,
                  scenario: str,
                  include_few_shot_examples: bool,
                  prompt: PromptTemplateSpec,
                  artifact_id: str) -> str:
    assert metric in SUPPORTED_METRICS, f"Unsupported metric: {metric}. Supported metrics: {SUPPORTED_METRICS}"
    assert reference_model in SUPPORTED_MODELS, f"Unsupported reference model: {reference_model}. Supported models: {SUPPORTED_MODELS}"
    #assert all(model in SUPPORTED_MODELS for model in targets.keys()), f"Unsupported target models: {targets}. Supported models: {SUPPORTED_MODELS}"
    #assert all(bool(SEMVER_PATTERN.match(value)) for value in targets.values()), f"Prompt target has to match '{SEMVER_PATTERN}'"
    input_parameters = [
        ParameterBinding(key="dataset", value=dataset_path),
        ParameterBinding(key="optimizationMetric", value=metric),
        ParameterBinding(key="basePrompt", value=f'{scenario}/{prompt["name"]}:{prompt["version"]}'),
        ParameterBinding(key="baseModel", value=reference_model),
        ParameterBinding(key="targetModels", value=','.join(targets.keys())),
        ParameterBinding(key="includeFewShotExamples", value=str(include_few_shot_examples).lower()),
        ParameterBinding(key="targetPromptMapping", value=",".join([f"{k}={v}" for k, v in targets.items()]))
    ]
    

    input_artifacts = [InputArtifactBinding(key="prompt-data", artifact_id = artifact_id)]

    response = client.ai_core_client.configuration.create(
        name = "facility-classification-optimization", # custom name of configuration
        scenario_id = "genai-optimizations", # value from workflow
        executable_id = "genai-optimizations", # value from workflow
        resource_group = "default",
        parameter_bindings = input_parameters,
        input_artifact_bindings = input_artifacts
    )

    return response.id



In [83]:
from rich.console import Console
from rich.table import Table

def fetch_results(execution_id):
    response = client.ai_core_client.execution.get(execution_id = execution_id)
    if response.status.name not in {'DEAD', 'COMPLETED'}:
        raise RuntimeError('Execution not finished!')
    path = f"default/{execution_id}/result-data/results.json"
    encoded_path = quote(path, safe="")
    url = f"{client.ai_core_client.base_url}/lm/dataset/files/{encoded_path}"
    headers = {
        **client.request_header,
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()# results = response.json()


def print_results_with_prompts(result):
    print_prompt_template(PromptTemplateSpec.from_optimizer_result(result["origin_model"]), addition='Baseline Prompt')
    origin_model = result["origin_model"]
    table = Table(title="Performance")
    table.add_column("Model", justify="right", style="cyan", no_wrap=True)
    table.add_column("Pre Optimization", style="magenta")
    table.add_column("Post Optimization", justify="right", style="green")
    table.add_row(origin_model["model_name"], f'{origin_model["score"]:.3f}', "n/a - reference run")
    for m in result["target_models"]:
        if m['result_status'] not in {'failed'}:
            table.add_row(m["model_name"], f'{m["pre_optimization_score"]:.3f}', f'{m["post_optimization_score"]:.3f}')
    for m in result["target_models"]:
        print_prompt_template(PromptTemplateSpec.from_optimizer_result(m), addition=m['model_name'])
    console.print(table)

def print_results_only(result):
    origin_model = result["origin_model"]
    table = Table(title="Performance")
    table.add_column("Model", justify="right", style="cyan", no_wrap=True)
    table.add_column("Pre Optimization", style="magenta")
    table.add_column("Post Optimization", justify="right", style="green")
    table.add_row(origin_model["model_name"], f'{origin_model["score"]:.3f}', "n/a - reference run")
    for m in result["target_models"]:
        if m['result_status'] not in {'failed'}:
            table.add_row(m["model_name"], f'{m["pre_optimization_score"]:.3f}', f'{m["post_optimization_score"]:.3f}')
    console.print(table)



def print_logs(execution_id):
    try:
        fetch_results(execution_id, file_name='result-data/log.txt')
    except HTTPError:
        console.print(Panel(Text(f"No logs found for {execution_id} ❌", style="bold red"), border_style="red"))
    else:
        console.print(Panel(Text("Run completed successfully ✅", style="bold green"), border_style="green"))

# Demo Scenario

Our Scenario for this hands-on session is an existing AI use-case tha supports a facility management in analysing messages the team receives from employees about issues in the office. 
For this sceraio we have an existing prompt as well as data sets for optimization and evaluation with messages and golden answers.

## Step 1: Familiarize youself with the Demo Data

Let's get familiar with the prompt and the data. You can find the basic prompt in the `prompts` folder stored as `facility_prompt_basic.yaml`. 

In [39]:
print("Basic prompt:") 
basic_prompt = open("../../data/prompts/facility_prompt_basic.yaml", "r").read()
print(basic_prompt) 

Basic prompt:

system: |-
    You are a helpful assistant

user: |-
    Giving the following message:
    ---
    {{?input}}
    ---
    Extract and return a json with the follwoing keys and values:
    - "urgency" as one of `high`, `medium`, `low`
    - "sentiment" as one of `negative`, `neutral`, `positive`
    - "categories" Create a dictionary with categories as keys and boolean values (True/False), 
    where the value indicates whether the category is one of the best matching support category tags from: 
    `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`, 
    `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`, 
    `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`, 
    `facility_management_issues`

    Your complete message should be a valid json string that can be read directly and only contain the keys 
    mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.

 Regarding golden sample data we have two data sets one will be used for evaluations and one for optimizations they can be found in the `data/evaluation` and the `data/optimization` folder respectively.

In [40]:
print("Example data entry:")
with open("../../data/datasets/facility_eval_data.json", "r") as input_file:
    head = [next(input_file) for _ in range(25)]
print(head)

Example data entry:

[
    '[\n',
    '        {\n',
    '                "input": "Subject: Assistance Needed with Facility Management Coordination\\n\\nHi Support 
Team,\\n\\nI hope this message finds you well. My name is [Sender], and I\'ve been utilizing Facility Solutions 
Company for our office building\'s management for the past year. Overall, I\'ve been quite satisfied with the 
services provided, but I\\u2019ve encountered an issue that I need your help with.\\n\\nRecently, I\'ve noticed 
some inconsistencies in the coordination of our space utilization and security measures. Specifically, there have 
been a few instances where the security protocols were not followed correctly, and the space allocation for our new
team members hasn\'t been managed as efficiently as before. This has caused some disruptions in our daily 
operations.\\n\\nI haven\'t taken any steps to address this issue internally yet, as I believe it would be best 
handled by your team of experts. Could you please look into this matter and provide a solution to ensure that our 
facility management runs smoothly again?\\n\\nThank you for your attention to this matter. I look forward to your 
prompt assistance.\\n\\nBest regards,\\n[Sender]",\n',
    '                "reference": "{\\"categories\\": [\\"facility_management_issues\\", 
\\"quality_and_safety_concerns\\"], \\"sentiment\\": \\"neutral\\", \\"urgency\\": \\"medium\\"}",\n',
    '                "json_schema_value": {\n',
    '                        "$schema": "http://json-schema.org/draft-07/schema#",\n',
    '                        "type": "object",\n',
    '                        "properties": {\n',
    '                                "categories": {\n',
    '                                        "type": "array"\n',
    '                                },\n',
    '                                "sentiment": {\n',
    '                                        "type": "integer"\n',
    '                                },\n',
    '                                "urgency": {\n',
    '                                        "type": "string"\n',
    '                                }\n',
    '                        },\n',
    '                        "required": [\n',
    '                                "categories",\n',
    '                                "sentiment",\n',
    '                                "urgency"\n',
    '                        ]\n',
    '                }\n',
    '        },\n'
]

_We should have the facility-prompt yaml the test and the train data-set in the s3 store already.
Does the store need the setup from the evaluation notebooks (eval- runs-prompts -testdata-dataset)?_
_No orchestration config necessary - working directly with templates is possible directly_

## Step 2: Let's evaluate the current prompt with the dataset

Let's evaluate the current prompt with the evaluation dataset for a given model.


### Step 2.1: Upload the prompt to the prompt registry

First let's evaluate our prompt and the existing placeholders.

In [35]:
base_prompt_template = "../../data/prompts/facility_prompt_basic.yaml" # local path to the prompt template or Prompt Repository identifier


prompt = load_prompt_template(base_prompt_template) # .escape_curly_brackets() if validation fails.
print_prompt_template(prompt)
print(f"Prompt template loaded successfully. Placeholders found are: {prompt.placeholders}")
assert validate_prompt(prompt)


╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ You are a helpful assistant                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── User Message ──────────────────────────────────────────────────╮
│ Giving the following message:                                                                                   │
│ ---                                                                                                             │
│ {{?input}}                                                                                                      │
│ ---                                                                                                             │
│ Extract and return a json with the follwoing keys and values:                                                   │
│ - "urgency" as one of `high`, `medium`, `low`                                                                   │
│ - "sentiment" as one of `negative`, `neutral`, `positive`                                                       │
│ - "categories" Create a dictionary with categories as keys and boolean values (True/False),                     │
│ where the value indicates whether the category is one of the best matching support category tags from:          │
│ `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`,                     │
│ `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`,             │
│ `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`,            │
│ `facility_management_issues`                                                                                    │
│                                                                                                                 │
│ Your complete message should be a valid json string that can be read directly and only contain the keys         │
│ mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Prompt template loaded successfully. Placeholders found are: {'input'}

Now let's upload the prompt to the prompt registry so that we can reference it during evaluation and optimization.

In [89]:
base_template = load_prompt_template(base_prompt_template)
base_prompt_template_registry = groupID+"-facility-classification-base:0.0.1"  # name:version for the template in the registry
prompt_template_name_registry, _, prompt_template_version = base_prompt_template_registry.partition(":")

prompt = push_prompt_template(prompt_template=base_template,
                              prompt_template_name_registry=prompt_template_name_registry,
                              prompt_template_version=prompt_template_version,
                              scenario=scenario,
                              update=True
)

print(f"Prompt present in registry under id {prompt['id']}")

print('\n\n=== Base Prompt ===')
print_prompt_template(prompt["id"])

Prompt present in registry under id a6768190-6f4f-4f64-b3a5-ab95e0968c37

=== Base Prompt ===

╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ You are a helpful assistant                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── User Message ──────────────────────────────────────────────────╮
│ Giving the following message:                                                                                   │
│ ---                                                                                                             │
│ {{?input}}                                                                                                      │
│ ---                                                                                                             │
│ Extract and return a json with the follwoing keys and values:                                                   │
│ - "urgency" as one of `high`, `medium`, `low`                                                                   │
│ - "sentiment" as one of `negative`, `neutral`, `positive`                                                       │
│ - "categories" Create a dictionary with categories as keys and boolean values (True/False),                     │
│ where the value indicates whether the category is one of the best matching support category tags from:          │
│ `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`,                     │
│ `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`,             │
│ `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`,            │
│ `facility_management_issues`                                                                                    │
│                                                                                                                 │
│ Your complete message should be a valid json string that can be read directly and only contain the keys         │
│ mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Step 2.2: Upload the data set to the prepared object store

Now let's validate our data set used for the evaluation and upload it to the object store.

In [55]:
dataset_secret="default" # secret name in the object store <you want to use to store the dataset
dataset_local_path="../../data/datasets/facility_eval_data.json" # local path to the dataset
dataset_remote_path=groupID+"/testdata/facility-classification-eval.json" # remote path in the object store to store the dataset
artifact_root_path="ai://default/"+groupID+"/"
dataset_path = upload_file(
    secret=dataset_secret,
    local_path=dataset_local_path,
    remote_path=dataset_remote_path,
    overwrite=True
    )
artifact = register_artifact(
    artifact_name="genai-eval-simplified-test-data",
    artifact_kind=Artifact.Kind.OTHER,
    artifact_root_path=artifact_root_path,
    scenario="genai-evaluations",
    description="Artifact for evaluation flow"
)

print(f"Dataset uploaded to {artifact.url}/{dataset_path} -> Artifact ID: {artifact.id}")

Dataset uploaded to ai://default/T-63//ai://default/T-63/testdata -> Artifact ID: 
65e6df78-6255-4786-bbd2-af9110589bd2

### Step 2.3: Configure and start the Evaluation



#### Step 2.3.1: Check available models

#### Step 2.3.2 Chech available metrics

#### Step 2.3.3: Configure the Evaluation

In [60]:
# Manual Selection of Metrics and Models
selected_metrics_str = "JSON Schema Match"
selected_models_str = targetModel #model name as listed in models endpoint

test_data_path = f"testdata/facility-classification-eval.json" # specify the test data path here. For the full folder just specifying testdata will work
test_datasets = json.dumps({'path': test_data_path, 'type': 'json'})
metrics_list = selected_metrics_str
models_list = selected_models_str
print(f"Selected metrics: {metrics_list}")
print(f"Selected models: {models_list}")
variable_mapping = json.dumps({'json_schema_match/json_schema':'data/json_schema_value','prompt/input': 'data/input'}) # to map the question prompt variable to the entry in dataset.
orchestration_deployment_url = orchestration_deployment.deployment_url # need to specify this to use a specific deployment id
repetitions = "1"

Selected metrics: JSON Schema Match

Selected models: gpt-4o-mini:latest

In [61]:
#  creating an AICORE Configuration
import requests

request_body = {
    "name": "genai-eval-conf",
    "scenarioId": "genai-evaluations",
    "executableId": "genai-evaluations-simplified",
    "inputArtifactBindings": [
        {
            "key": "datasetFolder",
            "artifactId": artifact.id
        }
    ],
    "parameterBindings": [
        {
            "key": "repetitions",
            "value": repetitions
        },
        {
            "key": "orchestrationDeploymentURL",
            "value": orchestration_deployment_url
        },
        {
            "key": "metrics",
            "value": metrics_list
        },
        {
            "key": "testDataset",
            "value": test_datasets
        },
        {
            "key": "variableMapping",
            "value": variable_mapping
        },
        {
            "key": "promptTemplate",
            "value": prompt["id"]
        },
        {
            "key": "models",
            "value": models_list
        },
        {
            "key": "debugMode",
            "value": "ON"
        }
    ]
}

print(request_body )

def create_aicore_configuration():
    headers = headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    #GET_CONFIGURATIONS_ENDPOINT = '/v2/lm/configurations'
    request_url = client.ai_core_client.base_url + "/lm/configurations"
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print(response)
        if(response.status_code != 201):
            raise
        result = response.json()
        print(result)
        return result['id']
    except:
       print("Error occurred while attempting to create a Configuration")
       print(response.text)
       raise
        
configuration_id = create_aicore_configuration()

{
    'name': 'genai-eval-conf',
    'scenarioId': 'genai-evaluations',
    'executableId': 'genai-evaluations-simplified',
    'inputArtifactBindings': [{'key': 'datasetFolder', 'artifactId': '65e6df78-6255-4786-bbd2-af9110589bd2'}],
    'parameterBindings': [
        {'key': 'repetitions', 'value': '1'},
        {
            'key': 'orchestrationDeploymentURL',
            'value': 'https://api.ai.staging.eu-west-1.mlf-aws-dev.com/v2/inference/deployments/d15746656296bd06'
        },
        {'key': 'metrics', 'value': 'JSON Schema Match'},
        {'key': 'testDataset', 'value': '{"path": "testdata/facility-classification-eval.json", "type": "json"}'},
        {
            'key': 'variableMapping',
            'value': '{"json_schema_match/json_schema": "data/json_schema_value", "prompt/input": "data/input"}'
        },
        {'key': 'promptTemplate', 'value': 'c37d3749-8e37-4b6a-a067-168c49bc8469'},
        {'key': 'models', 'value': 'gpt-4o-mini:latest'},
        {'key': 'debugMode', 'value': 'ON'}
    ]
}

<Response [201]>

{'id': '46235a44-845a-4a68-9d48-ecd533b03386', 'message': 'Configuration created'}

#### Step 2.3.4: Start the Evaluation

In [62]:
# create an execution with the created configuration.

import requests

def create_execution():
    headers = headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    #GET_EXECUTIONS_ENDPOINT = '/v2/lm/executions'
    request_url = client.ai_core_client.base_url + "/lm/executions"
    request_body = {"configurationId" : configuration_id} 
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print("response received is ", response)
        result = response.json()
        print(result)
        return result['id']
    except:
        print("Error occurred while attempting to create an execution")
        raise
 

execution_id = create_execution()

response received is  <Response [202]>

{'id': 'e607b094e229abf3', 'message': 'Execution scheduled', 'status': 'UNKNOWN', 'targetStatus': 'COMPLETED'}

### 2.4: Check the results of your evaluation

#### 2.4.1 Check the status of the evaluation

In [64]:
poll_status(execution_id, client)

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Run completed successfully ✅                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

'COMPLETED'

#### 2.4.2 Check logs of your evaluation

#### 2.4.3 Check results of your evalation

## Step 3: Let's improve the prompt with the help of our prompt Optimizer

### 3.1 Upload the optimization data

#### 3.1.1 Upload the data set to the prepared object store
Now let's upload the data set used for the optimization.

In [116]:
dataset_secret="default" # secret name in the object store <you want to use to store the dataset
dataset_local_path="../../data/datasets/facility_optim_data.json" # local path to the dataset
dataset_remote_path=groupID+"/testdata/facility-classification-optim.json" # remote path in the object store to store the dataset
dataset_path = upload_file(
    secret=dataset_secret,
    local_path=dataset_local_path,
    remote_path=dataset_remote_path,
    overwrite=True
    )

print(f"Dataset uploaded to {dataset_path}")

Dataset uploaded to ai://default/T-63/testdata

#### 3.1.2 Create artifact for optimization data

In [117]:
artifact_root_path="ai://default/"+groupID+"/"
artifact = register_artifact(
    artifact_name="facility-classification-optimization-data",
    artifact_kind=Artifact.Kind.DATASET,
    artifact_root_path=artifact_root_path,
    scenario="genai-optimizations",
    description="Artifact for prompt optimization"
)

print("-> Artifact ID:" + artifact.id)

-> Artifact ID:7c7a6084-84da-4f95-84f6-c21ddfdc7e3f

#### 3.2 Configure and start the optimization

### 3.2.1 Configure the optimization

In [118]:
scenario = "prompt-optimization-demo"

base_prompt_template_registry = "AI-161-"+groupID+"/"+groupID+"-facility-classification-base:0.0.1"  # name:version for the template in the registry

dataset_secret="default" # secret name in the object store <you want to use to store the dataset
dataset_remote_path="testdata/facility-classification-optim.json" # remote path in the object store to store the dataset


include_few_shot_examples = True # whether to include few shot examples in the prompt

reference_model = "gpt-4o:2024-08-06"
# Dictionary of models to optimize with their corresponding prompt template names under which the optimized prompt should be stored in the registry
targets = {
    targetModel : groupID+"-facility-classification-optim:0.0.1"
}
# Metric to use for optimization
metric = "JSON_Match" # Semantic

configuration_id = create_optim_config(metric=metric,
                                 reference_model=reference_model,
                                 targets=targets,
                                 dataset_path="testdata/facility-classification-optim.json",
                                 scenario=scenario,
                                 include_few_shot_examples=include_few_shot_examples,
                                 prompt=prompt,
                                 artifact_id=artifact.id)

#### 3.2.2 Start the optimization

In [119]:
response = client.ai_core_client.execution.create(
    configuration_id = configuration_id, # Change this value.
    resource_group = "default"
)

execution_id = response.id
print('Execution started with ID:', execution_id)

Execution started with ID: ef6844185c528f27

### 3.3 Check your optimization

#### 3.3.1 Check the status of the optimization

In [120]:
poll_status(execution_id, client)

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Run completed successfully ✅                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

'COMPLETED'

#### 3.3.2 Check logs of your optimization

In [88]:
p_stat = poll_status(execution_id, client)
if  p_stat == 'DEAD' or p_stat == 'COMPLETED':
    print("todo print logs function")

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Run is DEAD 💀                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

todo print logs function

#### 3.3.3 Check results of your optimization

In [ ]:
if poll_status(execution_id, client) == 'COMPLETED':
    results = fetch_results(execution_id)
    print_results_with_prompts(results)

## Step 4: Let's evaluate the optimized prompt with the evaluation service and our evaluation dataset

Let's evaluate the curoptimized prompt with the evaluation dataset to compare pre and post optimization scores.


### Step 4.1: Configure and start the Evaluation



#### Step 4.1.1: Configure the Evaluation

In [107]:
# Manual Selection of Metrics and Models
selected_metrics_str = "JSON Schema Match"
selected_models_str = targetModel #model name as listed in models endpoint

test_data_path = f"testdata/facility-classification-eval.json" # specify the test data path here. For the full folder just specifying testdata will work
test_datasets = json.dumps({'path': test_data_path, 'type': 'json'})
metrics_list = selected_metrics_str
models_list = selected_models_str
print(f"Selected metrics: {metrics_list}")
print(f"Selected models: {models_list}")
variable_mapping = json.dumps({'json_schema_match/json_schema':'data/json_schema_value','prompt/input': 'data/input'}) # to map the question prompt variable to the entry in dataset.
orchestration_deployment_url = orchestration_deployment.deployment_url # need to specify this to use a specific deployment id
repetitions = "1"

optim_prompt=fetch_prompt_template("AI161-"+groupID+"/"+groupID+"-facility-classification-optim/:/0.0.1")
print(optim_prompt)

Selected metrics: JSON Schema Match

Selected models: gpt-4o-mini:latest

HTTPError: 400 Client Error: Bad Request for url: https://api.ai.staging.eu-west-1.mlf-aws-dev.com/v2/lm/promptTemplates?name=T-63-facility-classification-optim%2F&version=%2F0.0.1&scenario=AI161-T-63&includeSpec=True

In [ ]:
#  creating an AICORE Configuration
import requests

request_body = {
    "name": "genai-eval-conf",
    "scenarioId": "genai-evaluations",
    "executableId": "genai-evaluations-simplified",
    "inputArtifactBindings": [
        {
            "key": "datasetFolder",
            "artifactId": artifact.id
        }
    ],
    "parameterBindings": [
        {
            "key": "repetitions",
            "value": repetitions
        },
        {
            "key": "orchestrationDeploymentURL",
            "value": orchestration_deployment_url
        },
        {
            "key": "metrics",
            "value": metrics_list
        },
        {
            "key": "testDataset",
            "value": test_datasets
        },
        {
            "key": "variableMapping",
            "value": variable_mapping
        },
        {
            "key": "promptTemplate",
            "value": optim_prompt["id"]
        },
        {
            "key": "models",
            "value": models_list
        },
        {
            "key": "debugMode",
            "value": "ON"
        }
    ]
}

print(request_body )

def create_aicore_configuration():
    headers = headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    #GET_CONFIGURATIONS_ENDPOINT = '/v2/lm/configurations'
    request_url = client.ai_core_client.base_url + "/lm/configurations"
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print(response)
        if(response.status_code != 201):
            raise
        result = response.json()
        print(result)
        return result['id']
    except:
       print("Error occurred while attempting to create a Configuration")
       print(response.text)
       raise
        
configuration_id = create_aicore_configuration()

{
    'name': 'genai-eval-conf',
    'scenarioId': 'genai-evaluations',
    'executableId': 'genai-evaluations-simplified',
    'inputArtifactBindings': [{'key': 'datasetFolder', 'artifactId': '65e6df78-6255-4786-bbd2-af9110589bd2'}],
    'parameterBindings': [
        {'key': 'repetitions', 'value': '1'},
        {
            'key': 'orchestrationDeploymentURL',
            'value': 'https://api.ai.staging.eu-west-1.mlf-aws-dev.com/v2/inference/deployments/d15746656296bd06'
        },
        {'key': 'metrics', 'value': 'JSON Schema Match'},
        {'key': 'testDataset', 'value': '{"path": "testdata/facility-classification-eval.json", "type": "json"}'},
        {
            'key': 'variableMapping',
            'value': '{"json_schema_match/json_schema": "data/json_schema_value", "prompt/input": "data/input"}'
        },
        {'key': 'promptTemplate', 'value': 'c37d3749-8e37-4b6a-a067-168c49bc8469'},
        {'key': 'models', 'value': 'gpt-4o-mini:latest'},
        {'key': 'debugMode', 'value': 'ON'}
    ]
}

<Response [201]>

{'id': '46235a44-845a-4a68-9d48-ecd533b03386', 'message': 'Configuration created'}

#### Step 4.1.2: Start the Evaluation

In [ ]:
# create an execution with the created configuration.

import requests

def create_execution():
    headers = headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    #GET_EXECUTIONS_ENDPOINT = '/v2/lm/executions'
    request_url = client.ai_core_client.base_url + "/lm/executions"
    request_body = {"configurationId" : configuration_id} 
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print("response received is ", response)
        result = response.json()
        print(result)
        return result['id']
    except:
        print("Error occurred while attempting to create an execution")
        raise
 

post_eval_id = create_execution()

response received is  <Response [202]>

{'id': 'e607b094e229abf3', 'message': 'Execution scheduled', 'status': 'UNKNOWN', 'targetStatus': 'COMPLETED'}

### 4.2: Check the results of your post evaluation

#### 4.2.1 Check the status of the post evaluation

In [ ]:
poll_status(post_eval_id, client)

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Run is DEAD 💀                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

'DEAD'

#### 2.4.2 Check logs of your evaluation

#### 2.4.3 Check results of your evalation

In [ ]:
# create an execution with the created configuration.

import requests

def create_execution():
    headers = headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    #GET_EXECUTIONS_ENDPOINT = '/v2/lm/executions'
    request_url = client.ai_core_client.base_url + "/lm/executions"
    request_body = {"configurationId" : configuration_id} 
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print("response received is ", response)
        result = response.json()
        print(result)
        return result['id']
    except:
        print("Error occurred while attempting to create an execution")
        raise
 

execution_id = create_execution()

response received is  <Response [202]>

{'id': 'e607b094e229abf3', 'message': 'Execution scheduled', 'status': 'UNKNOWN', 'targetStatus': 'COMPLETED'}